# NONLINEAR_SYSTEM

## Overview
Solves systems of nonlinear equations by finding values for variables that satisfy all equations simultaneously. This function enables business users to solve complex, highly nonlinear relationships directly in Excel, leveraging the CasADi package for advanced root-finding. It is suitable for both under- and over-determined systems and is accessible via the Boardflare Python for Excel add-in.

## Usage
To use the `NONLINEAR_SYSTEM` function in Excel, enter it as a formula in a cell, specifying your equations and initial guesses:

```excel
=NONLINEAR_SYSTEM(equations, variables)
```

## Arguments
| Argument   | Type           | Required | Description                                                              | Example                       |
|------------|----------------|----------|--------------------------------------------------------------------------|-------------------------------|
| equations  | 2D list string | Yes      | Symbolic expressions for the system of equations to solve.               | [["x**2 + y**2 - 1", "x - y"]] |
| variables  | 2D list float  | Yes      | Initial guesses for the variables.                                       | [[0.5, 0.5]]                  |

## Returns
| Type         | Description                                                      | Example         |
|--------------|------------------------------------------------------------------|-----------------|
| list[list[float]]| The solution for the variables that satisfy the system.          | [[0.707, 0.707]]|
| str          | Error message if the solver fails or input is invalid.           | "No solution found: system is inconsistent." |

## Examples
**Chemical Equilibrium**
*Business context*: A chemist needs to solve for concentrations of two substances at equilibrium, given two nonlinear equations representing the chemical reactions.

Excel usage:
```excel
=NONLINEAR_SYSTEM({"x**2 + y**2 - 1", "x - y"}, {0.5, 0.5})
```
*Expected outcome*: Returns the concentrations of x and y that satisfy both equations.

**Engineering Circuit Analysis**
*Business context*: An engineer wants to find the voltages in a nonlinear circuit described by two equations.

Excel usage:
```excel
=NONLINEAR_SYSTEM({"x**3 + y - 1", "x + y**3 - 1"}, {0.7, 0.7})
```
*Expected outcome*: Returns the voltages x and y that solve the system.

## Limitations
- All expressions must use valid Python syntax and variable names.
- Only supports scalar or 2D list arguments of type float or string.
- Some nonlinear systems may not have a solution or may be sensitive to initial guesses.

## Benefits
Excel's built-in Solver can solve some systems of equations, but it is limited in handling large, nonlinear, or symbolically defined systems. This Python function, powered by CasADi, allows for more flexible and scalable equation solving directly from Excel, with support for symbolic expressions and advanced solver options.

In [ ]:
import casadi as ca
import numpy as np
import re

def nonlinear_system(equations, variables):
    """
    Solves systems of nonlinear equations using CasADi's root-finding capabilities.

    Args:
        equations: 2D list of strings, symbolic expressions for the system (e.g., [["x**2 + y**2 - 1", "x - y"]]).
        variables: 2D list of floats, initial guesses for the variables (e.g., [[0.5, 0.5]]).

    Returns:
        list[list[float]]: The solution for the variables, or error message string.
    """
    try:
        if not (isinstance(equations, list) and len(equations) > 0 and all(isinstance(row, list) for row in equations)):
            return "equations must be a 2D list of strings."
        if not (isinstance(variables, list) and len(variables) > 0 and isinstance(variables[0], list)):
            return "variables must be a 2D list of floats."
        eqn_list = [eq for row in equations for eq in row]
        n_eqns = len(eqn_list)
        var_vals = variables[0]
        n_vars = len(var_vals)
        if n_vars != n_eqns:
            return f"Error: number of variables ({n_vars}) must match number of equations ({n_eqns})."
        # Use a single vector variable x for all unknowns
        x = ca.MX.sym('x', n_vars)
        # Map variable names (x, y, z, ...) to x[0], x[1], ...
        # Always use the first n_vars lowercase letters as variable names
        var_names = [chr(ord('x') + i) for i in range(n_vars)]
        def substitute_vars(expr, var_names):
            # Replace all variable names with x[i] in order
            for i, name in enumerate(var_names):
                expr = re.sub(rf'(?<![A-Za-z0-9_]){name}(?![A-Za-z0-9_])', f'x[{i}]', expr)
            return expr
        local_dict = {'x': x, 'ca': ca}
        f_list = []
        for eq in eqn_list:
            try:
                eq_sub = substitute_vars(eq, var_names)
                f_list.append(eval(eq_sub, local_dict))
            except Exception as e:
                return f"Invalid equation: {eq}: {str(e)}"
        F = ca.vertcat(*f_list)
        F_func = ca.Function('F_func', [x], [F])
        from casadi import rootfinder
        try:
            solver = rootfinder('solver', 'newton', F_func, {})
            sol = solver(np.array(var_vals, dtype=float))
        except Exception as e:
            return f"Solver failed: {str(e)}"
        # Always return a list of lists of floats for valid solutions
        if hasattr(sol, 'full'):
            return [sol.full().flatten().tolist()]
        elif isinstance(sol, np.ndarray):
            return [sol.flatten().tolist()]
        elif isinstance(sol, (list, tuple)):
            return [list(sol)]
        else:
            return "Error during CasADi calculation: Unexpected result type."
    except ca.CasadiException as e:
        return f"Error during CasADi calculation: {e}"
    except Exception as e:
        return str(e)

In [ ]:
%pip install -q ipytest
import ipytest
ipytest.autoconfig()

def test_demo_chem_equilibrium():
    equations = [["x**2 + y**2 - 1", "x - y"]]
    variables = [[0.5, 0.5]]
    result = nonlinear_system(equations, variables)
    assert isinstance(result, list)
    assert len(result) == 1
    assert all(isinstance(row, list) or isinstance(row, float) for row in result)

def test_demo_circuit_analysis():
    equations = [["x**3 + y - 1", "x + y**3 - 1"]]
    variables = [[0.7, 0.7]]
    result = nonlinear_system(equations, variables)
    assert isinstance(result, list)
    assert len(result) == 1
    assert all(isinstance(row, list) or isinstance(row, float) for row in result)

def test_error_invalid_equation():
    equations = [["x**2 +"]]
    variables = [[0.5, 0.5]]
    result = nonlinear_system(equations, variables)
    assert isinstance(result, str) and ("error" in result.lower() or "must be" in result.lower() or "invalid" in result.lower())

def test_error_mismatched_vars():
    equations = [["x**2 + y**2 - 1", "x - y"]]
    variables = [[0.5]]
    result = nonlinear_system(equations, variables)
    assert isinstance(result, str) and ("error" in result.lower() or "must be" in result.lower() or "invalid" in result.lower())

ipytest.run()

In [ ]:
# Interactive Demo
import gradio as gr

examples = [
    [
        [["x**2 + y**2 - 1", "x - y"]],
        [[0.5, 0.5]]
    ],
    [
        [["x**3 + y - 1", "x + y**3 - 1"]],
        [[0.7, 0.7]]
    ]
]

demo = gr.Interface(
    fn=nonlinear_system,
    inputs=[
        gr.Dataframe(headers=["Equation 1", "Equation 2"], label="Equations", row_count=1, col_count=2, type="array",
                     value=[["x**2 + y**2 - 1", "x - y"]]),
        gr.Dataframe(headers=["Variable 1", "Variable 2"], label="Initial Guesses", row_count=1, col_count=2, type="array", 
                     value=[[0.5, 0.5]]),
    ],
    outputs=gr.Dataframe(headers=["Solution"], label="Solution"),
    examples=examples,
    description="Solve a system of nonlinear equations using CasADi root-finding.",
    flagging_mode="never",
)
demo.launch()